In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

In [8]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:Path
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
from mlproject.constant import *
from mlproject.utils.common import read_yaml_file,create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=config_file_path,
                params_filepath=params_file_path,
                schema_filepath=schema_file_path):
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)
        self.schema=read_yaml_file(schema_filepath)
        
        create_directories([self.config.artifacts_root]) 
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.config.target_column
        
        create_directories([config.root_dir])
        
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.target_column,
        )
        return model_trainer_config

In [10]:
import os
from mlproject import logger
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
class ModelTrainer:
    def __init__(self,config=ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)
        
        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        
        lr=ElasticNet(alpha=self.config.alpha,
                    L1_ratio=self.config.l1_ratio,
                    random_state=42)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [13]:
try:
    config=ConfigurationManager()
    model_train_config=config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_train_config)
    model_train_config. train()
except Exception as e:
    raise 2

[2024-08-12 16:58:10,916: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-08-12 16:58:10,927: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-08-12 16:58:10,931: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-08-12 16:58:10,934: INFO: common: created directory at artifacts:]


TypeError: exceptions must derive from BaseException

In [17]:
def test(x, y=[]):
	y.append(x)
	return y
print(test(1))
print(test(2))

[1]
[1, 2]
